In [4]:
import numpy as np
import tensorflow as tf

In [ ]:
x_data = [
    [1, 2, 1, 1],
    [2, 1, 3, 2],
    [3, 1, 3, 4],
    [4, 1, 5, 5],
    [1, 7, 5, 5],
    [1, 2, 5, 6],
    [1, 6, 6, 6],
    [1, 7, 7, 7]
]

y_data = [
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0],
    [1, 0, 0],
    [1, 0, 0]
]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)
nb_classes = 3 #num classes

In [23]:
#Weight and bias setting
W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

print(variables)

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.30405334, -0.5916206 ,  0.42229214],
       [-0.3615684 , -2.2092047 , -0.16092242],
       [-0.06034185,  0.21559335, -0.30302727],
       [-1.6994094 , -0.78747946, -0.269722  ]], dtype=float32)>, <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.576658  ,  0.12498101, -0.49841753], dtype=float32)>]


In [26]:
#softmax
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)
print(hypothesis(x_data))

tf.Tensor(
[[8.28621015e-02 1.02148438e-02 9.06922996e-01]
 [1.95545889e-02 4.95877489e-02 9.30857658e-01]
 [5.77944098e-04 6.81107026e-03 9.92610991e-01]
 [1.09065346e-04 4.16722614e-03 9.95723724e-01]
 [2.90354452e-04 4.02944778e-07 9.99709189e-01]
 [1.88299848e-04 6.68813754e-03 9.93123591e-01]
 [1.08302360e-04 3.12792349e-06 9.99888539e-01]
 [2.70412202e-05 4.03747237e-07 9.99972582e-01]], shape=(8, 3), dtype=float32)


In [27]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[7.761228e-06 1.971161e-06 9.999902e-01]], shape=(1, 3), dtype=float32)


In [28]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(5.629736, shape=(), dtype=float32)


In [29]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x)

print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [32]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.23440558, -0.7308519 ,  0.96525747],
       [-1.6013482 , -1.2382005 ,  2.8395486 ],
       [-1.6066203 , -1.8457861 ,  3.4524064 ],
       [-1.6089689 , -1.9752971 ,  3.584266  ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.23703527, -0.36531588,  0.6023511 ], dtype=float32)>]


In [33]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1)
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 1.429455
Loss at epoch 100: 0.684122
Loss at epoch 200: 0.603690
Loss at epoch 300: 0.544143
Loss at epoch 400: 0.491925
Loss at epoch 500: 0.442847
Loss at epoch 600: 0.395228
Loss at epoch 700: 0.348353
Loss at epoch 800: 0.302390
Loss at epoch 900: 0.261288
Loss at epoch 1000: 0.240801
Loss at epoch 1100: 0.228672
Loss at epoch 1200: 0.217694
Loss at epoch 1300: 0.207695
Loss at epoch 1400: 0.198548
Loss at epoch 1500: 0.190148
Loss at epoch 1600: 0.182407
Loss at epoch 1700: 0.175250
Loss at epoch 1800: 0.168615
Loss at epoch 1900: 0.162447
Loss at epoch 2000: 0.156700


In [35]:
sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype=np.float32)
a = hypothesis(sample_data)
b = hypothesis(x_data)

print(a)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor([[0.00212328 0.08399386 0.9138828 ]], shape=(1, 3), dtype=float32)
tf.Tensor(
[[2.5140998e-06 9.9464261e-04 9.9900287e-01]
 [2.1232790e-03 8.3993860e-02 9.1388279e-01]
 [1.2857106e-07 1.6469246e-01 8.3530742e-01]
 [4.3477253e-06 8.4856659e-01 1.5142898e-01]
 [2.5416908e-01 7.3354334e-01 1.2287644e-02]
 [1.3336249e-01 8.6663187e-01 5.6918711e-06]
 [7.5360888e-01 2.4637255e-01 1.8623281e-05]
 [9.2081511e-01 7.9184562e-02 2.8493000e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
